1) This model uses gridsearchCV to get best hyper-parameters for the Random Forest Classifier

---



In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
path = r"/content/drive/My Drive/Colab Notebooks/DataHack - Analytics Vidhya /JantaHack: Machine Learning in Agriculture"
train = pd.read_csv(path + "/train.csv")
test = pd.read_csv(path + "/test.csv")
submission = pd.read_csv(path + "/sample_submission.csv")


In [5]:
train.shape, test.shape, submission.shape

((88858, 10), (59310, 9), (59310, 2))

**Filling the null values in Number_Weeks_Used column**

In [6]:
train['Number_Weeks_Used'] = train['Number_Weeks_Used'].fillna(train.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))
test['Number_Weeks_Used'] = test['Number_Weeks_Used'].fillna(test.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))

# Data Preprocessing

In [7]:
training_labels = train.iloc[:, -1]
X_train = train.iloc[:, 1:-1]
X_test = test.iloc[:, 1:]

In [8]:
X_train.shape, X_test.shape

((88858, 8), (59310, 8))

In [9]:
data = pd.concat([X_train, X_test])

**OneHotEncoding for "Pesticide_Use_Category" and "Season"**

In [10]:
columns_names_encod = data.columns[[3, 7]]
data = pd.get_dummies(data, columns=columns_names_encod)

In [11]:
data.shape

(148168, 12)

**Splitting the data back**

In [12]:
final_train = data[:train.shape[0]]
final_test = data[train.shape[0]:]

In [13]:
final_train.shape, final_test.shape

((88858, 12), (59310, 12))

**Scaling the numerical variables**

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

final_train.iloc[:, [0, 3, 4, 5]] = sc.fit_transform(final_train.iloc[:, [0, 3, 4, 5]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
final_test.iloc[:, [0, 3, 4, 5]] = sc.transform(final_test.iloc[:, [0, 3, 4, 5]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# Creating cross validation set

In [16]:
training_labels.value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split
X_training, X_cv, y_training, y_cv = train_test_split(final_train, training_labels, test_size =0.2, random_state=21)

# Training model on training dataset


In [18]:
# To check whether GPU is running or not
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

param_grid = {'bootstrap': [True],
              'max_depth': [8, 10, 12, 14],
              'min_samples_split': [8, 10, 12, 15],
              'n_estimators': [50, 70, 90, 120],
             }

grid_search = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_training, y_training)

final_model = grid_search.best_estimator_

cvres = grid_search.cv_results_
for mean_accuracy_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_accuracy_score, params)

y_cv_pred = final_model.predict(X_cv)
grid_search.best_params_

0.8416425024116169 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 8, 'n_estimators': 50}
0.8417691232743065 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 8, 'n_estimators': 70}
0.8415862430295121 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 8, 'n_estimators': 90}
0.8415581076985704 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 8, 'n_estimators': 120}
0.8414737017057448 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 10, 'n_estimators': 50}
0.841529969992938 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 10, 'n_estimators': 70}
0.8417691214932889 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 10, 'n_estimators': 90}
0.8415721759577138 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 10, 'n_estimators': 120}
0.8421489413369326 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 12, 'n_estimators': 50}
0.8417550550151631 {'bootstrap': True, 'max_depth': 8, 'min_samples_split': 12, 'n_estimators': 70}
0.8

{'bootstrap': True,
 'max_depth': 12,
 'min_samples_split': 15,
 'n_estimators': 70}

**Checking accuracy score for training dataset and cross validation set**

In [20]:
from sklearn.metrics import accuracy_score
#training_accuracy_score = accuracy_score(y_training, y_training_pred)
cv_accuracy_score = accuracy_score(y_cv, y_cv_pred)

#print(training_accuracy_score)
print(cv_accuracy_score)

0.8395791132117938


# Testing the model on test data

In [23]:
predictions_test = final_model.predict(final_test)
predictions_test = pd.Series(predictions_test)

frame = {'ID': test.ID, 
         'Crop_Damage': predictions_test}
submission = pd.DataFrame(frame)
submission.to_csv(path + '/Random_Forest_Classifier_3_submission.csv', index=False)

{'bootstrap': True,
 'max_depth': 10,
 'min_samples_split': 10,
 'n_estimators': 50}